In [100]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import logging

logging.basicConfig(level=logging.INFO)

logger = logging.getLogger(__name__)
'''
First pass at data modeling. 

Predicting each updrs score individually as a function of the peptide abundance data with some feature reduction. 
'''

'\nFirst pass at data modeling. \n\nPredicting each updrs score individually as a function of the peptide abundance data with some feature reduction. \n'

In [101]:
def smape(y_true, y_pred):
    smap = np.zeros(len(y_true))
    
    num = np.abs(y_true - y_pred)
    dem = ((np.abs(y_true) + np.abs(y_pred)) / 2)
    
    pos_ind = (y_true != 0)|(y_pred != 0)
    smap[pos_ind] = num[pos_ind] / dem[pos_ind]
    
    return 100 * np.mean(smap)

In [102]:
proteins = pd.read_csv('./data/train_proteins.csv')
print(proteins.shape)
proteins.head()

(232741, 5)


,visit_id,visit_month,patient_id,UniProt,NPX
0,55_0,0,55,O00391,11254.3
1,55_0,0,55,O00533,732430.0
2,55_0,0,55,O00584,39585.8
3,55_0,0,55,O14498,41526.9
4,55_0,0,55,O14773,31238.0


In [103]:
peptides = pd.read_csv('./data/train_peptides.csv')
print(peptides.shape)
peptides.head()

(981834, 6)


,visit_id,visit_month,patient_id,UniProt,Peptide,PeptideAbundance
0,55_0,0,55,O00391,NEQEQPLGQWHLS,11254.3
1,55_0,0,55,O00533,GNPEPTFSWTK,102060.0
2,55_0,0,55,O00533,IEIPSSVQQVPTIIK,174185.0
3,55_0,0,55,O00533,KPQSAVYSTGSNGILLC(UniMod_4)EAEGEPQPTIK,27278.9
4,55_0,0,55,O00533,SMEQNGPGLEYR,30838.7


In [104]:
clinical = pd.read_csv('./data/train_clinical_data.csv')
print(clinical.shape)
clinical.head()

(2615, 8)


,visit_id,patient_id,visit_month,updrs_1,updrs_2,updrs_3,updrs_4,upd23b_clinical_state_on_medication
0,55_0,55,0,10.0,6.0,15.0,NaN,NaN
1,55_3,55,3,10.0,7.0,25.0,NaN,NaN
2,55_6,55,6,8.0,10.0,34.0,NaN,NaN
3,55_9,55,9,8.0,9.0,30.0,0.0,On
4,55_12,55,12,10.0,10.0,41.0,0.0,On


In [105]:
mergred_protein_peptides = pd.merge(proteins, peptides, on=['visit_id', 'visit_month', 'patient_id', 'UniProt'])
print(mergred_protein_peptides.shape)

merged = pd.merge(mergred_protein_peptides, clinical, on=['visit_id', 'visit_month', 'patient_id'])
print(merged.shape)
merged.head()

(981834, 7)
(941744, 12)


,visit_id,visit_month,patient_id,UniProt,NPX,Peptide,PeptideAbundance,updrs_1,updrs_2,updrs_3,updrs_4,upd23b_clinical_state_on_medication
0,55_0,0,55,O00391,11254.3,NEQEQPLGQWHLS,11254.3,10.0,6.0,15.0,NaN,NaN
1,55_0,0,55,O00533,732430.0,GNPEPTFSWTK,102060.0,10.0,6.0,15.0,NaN,NaN
2,55_0,0,55,O00533,732430.0,IEIPSSVQQVPTIIK,174185.0,10.0,6.0,15.0,NaN,NaN
3,55_0,0,55,O00533,732430.0,KPQSAVYSTGSNGILLC(UniMod_4)EAEGEPQPTIK,27278.9,10.0,6.0,15.0,NaN,NaN
4,55_0,0,55,O00533,732430.0,SMEQNGPGLEYR,30838.7,10.0,6.0,15.0,NaN,NaN


In [106]:
pivoted = merged.pivot(index='visit_id', columns=['Peptide'], values='PeptideAbundance')
pivoted.fillna(0, inplace=True)
pivoted

Peptide,AADDTWEPFASGK,AAFGQGSGPIMLDEVQC(UniMod_4)TGTEASLADC(UniMod_4)K,AAFTEC(UniMod_4)C(UniMod_4)QAADK,AANEVSSADVK,AATGEC(UniMod_4)TATVGKR,AATVGSLAGQPLQER,AAVYHHFISDGVR,ADDKETC(UniMod_4)FAEEGK,ADDKETC(UniMod_4)FAEEGKK,ADDLGKGGNEESTKTGNAGSR,...,YSLTYIYTGLSK,YTTEIIK,YVGGQEHFAHLLILR,YVM(UniMod_35)LPVADQDQC(UniMod_4)IR,YVMLPVADQDQC(UniMod_4)IR,YVNKEIQNAVNGVK,YWGVASFLQK,YYC(UniMod_4)FQGNQFLR,YYTYLIMNK,YYWGGQYTWDMAK
visit_id,,,,,,,,,,,,,,,,,,,,,
10053_0,6580710.0,31204.4,7735070.0,0.0,0.00,0.0,46620.3,236144.0,0.0,0.0,...,202274.0,0.00,4401830.0,77482.6,583075.0,76705.7,104260.0,530223.0,0.0,7207.30
10053_12,6333510.0,52277.6,5394390.0,0.0,0.00,0.0,57554.5,108298.0,45885.4,0.0,...,201009.0,0.00,5001750.0,36745.3,355643.0,92078.1,123254.0,453883.0,49281.9,25332.80
10053_18,7129640.0,61522.0,7011920.0,35984.7,17188.00,19787.3,36029.4,708729.0,5067790.0,30838.2,...,220728.0,0.00,5424380.0,39016.0,496021.0,63203.6,128336.0,447505.0,52389.1,21235.70
10138_12,7404780.0,46107.2,10610900.0,0.0,20910.20,66662.3,55253.9,79575.5,6201210.0,26720.0,...,188362.0,9433.71,3900280.0,48210.3,328482.0,89822.1,129964.0,552232.0,65657.8,9876.98
10138_24,13788300.0,56910.3,6906160.0,13785.5,11004.20,63672.7,36819.8,34160.9,2117430.0,15645.2,...,206187.0,6365.15,3521800.0,69984.6,496737.0,80919.3,111799.0,0.0,56977.6,4903.09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8699_24,6312970.0,44462.7,12455000.0,11051.3,1163.18,43279.8,67743.5,325328.0,4666550.0,11038.5,...,289888.0,8615.27,8770410.0,33599.1,926094.0,118897.0,133682.0,571879.0,80268.3,54889.70
942_12,11289900.0,46111.7,11297300.0,0.0,13894.10,53755.0,40289.3,565112.0,0.0,26495.8,...,173259.0,4767.63,374307.0,35767.3,250397.0,65966.9,77976.8,486239.0,45032.7,0.00
942_24,10161900.0,32145.0,12388000.0,25869.2,17341.80,48625.5,45223.9,84448.0,4684800.0,23150.2,...,185428.0,5554.53,0.0,64049.8,479473.0,68505.7,74483.1,561398.0,52916.4,21847.60


In [107]:
df = pd.merge(clinical, pivoted, on='visit_id', how='right').set_index('visit_id')
df

,patient_id,visit_month,updrs_1,updrs_2,updrs_3,updrs_4,upd23b_clinical_state_on_medication,AADDTWEPFASGK,AAFGQGSGPIMLDEVQC(UniMod_4)TGTEASLADC(UniMod_4)K,AAFTEC(UniMod_4)C(UniMod_4)QAADK,...,YSLTYIYTGLSK,YTTEIIK,YVGGQEHFAHLLILR,YVM(UniMod_35)LPVADQDQC(UniMod_4)IR,YVMLPVADQDQC(UniMod_4)IR,YVNKEIQNAVNGVK,YWGVASFLQK,YYC(UniMod_4)FQGNQFLR,YYTYLIMNK,YYWGGQYTWDMAK
visit_id,,,,,,,,,,,,,,,,,,,,,
10053_0,10053,0,3.0,0.0,13.0,0.0,NaN,6580710.0,31204.4,7735070.0,...,202274.0,0.00,4401830.0,77482.6,583075.0,76705.7,104260.0,530223.0,0.0,7207.30
10053_12,10053,12,4.0,2.0,8.0,0.0,NaN,6333510.0,52277.6,5394390.0,...,201009.0,0.00,5001750.0,36745.3,355643.0,92078.1,123254.0,453883.0,49281.9,25332.80
10053_18,10053,18,2.0,2.0,0.0,0.0,NaN,7129640.0,61522.0,7011920.0,...,220728.0,0.00,5424380.0,39016.0,496021.0,63203.6,128336.0,447505.0,52389.1,21235.70
10138_12,10138,12,3.0,6.0,31.0,0.0,On,7404780.0,46107.2,10610900.0,...,188362.0,9433.71,3900280.0,48210.3,328482.0,89822.1,129964.0,552232.0,65657.8,9876.98
10138_24,10138,24,4.0,7.0,19.0,10.0,On,13788300.0,56910.3,6906160.0,...,206187.0,6365.15,3521800.0,69984.6,496737.0,80919.3,111799.0,0.0,56977.6,4903.09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8699_24,8699,24,11.0,10.0,13.0,2.0,On,6312970.0,44462.7,12455000.0,...,289888.0,8615.27,8770410.0,33599.1,926094.0,118897.0,133682.0,571879.0,80268.3,54889.70
942_12,942,12,5.0,2.0,25.0,0.0,NaN,11289900.0,46111.7,11297300.0,...,173259.0,4767.63,374307.0,35767.3,250397.0,65966.9,77976.8,486239.0,45032.7,0.00
942_24,942,24,2.0,3.0,23.0,NaN,NaN,10161900.0,32145.0,12388000.0,...,185428.0,5554.53,0.0,64049.8,479473.0,68505.7,74483.1,561398.0,52916.4,21847.60


In [108]:
df.insert(6, 'visit_month', df.pop('visit_month'))
df = df.drop('patient_id', axis=1)
df

,updrs_1,updrs_2,updrs_3,updrs_4,upd23b_clinical_state_on_medication,visit_month,AADDTWEPFASGK,AAFGQGSGPIMLDEVQC(UniMod_4)TGTEASLADC(UniMod_4)K,AAFTEC(UniMod_4)C(UniMod_4)QAADK,AANEVSSADVK,...,YSLTYIYTGLSK,YTTEIIK,YVGGQEHFAHLLILR,YVM(UniMod_35)LPVADQDQC(UniMod_4)IR,YVMLPVADQDQC(UniMod_4)IR,YVNKEIQNAVNGVK,YWGVASFLQK,YYC(UniMod_4)FQGNQFLR,YYTYLIMNK,YYWGGQYTWDMAK
visit_id,,,,,,,,,,,,,,,,,,,,,
10053_0,3.0,0.0,13.0,0.0,NaN,0,6580710.0,31204.4,7735070.0,0.0,...,202274.0,0.00,4401830.0,77482.6,583075.0,76705.7,104260.0,530223.0,0.0,7207.30
10053_12,4.0,2.0,8.0,0.0,NaN,12,6333510.0,52277.6,5394390.0,0.0,...,201009.0,0.00,5001750.0,36745.3,355643.0,92078.1,123254.0,453883.0,49281.9,25332.80
10053_18,2.0,2.0,0.0,0.0,NaN,18,7129640.0,61522.0,7011920.0,35984.7,...,220728.0,0.00,5424380.0,39016.0,496021.0,63203.6,128336.0,447505.0,52389.1,21235.70
10138_12,3.0,6.0,31.0,0.0,On,12,7404780.0,46107.2,10610900.0,0.0,...,188362.0,9433.71,3900280.0,48210.3,328482.0,89822.1,129964.0,552232.0,65657.8,9876.98
10138_24,4.0,7.0,19.0,10.0,On,24,13788300.0,56910.3,6906160.0,13785.5,...,206187.0,6365.15,3521800.0,69984.6,496737.0,80919.3,111799.0,0.0,56977.6,4903.09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8699_24,11.0,10.0,13.0,2.0,On,24,6312970.0,44462.7,12455000.0,11051.3,...,289888.0,8615.27,8770410.0,33599.1,926094.0,118897.0,133682.0,571879.0,80268.3,54889.70
942_12,5.0,2.0,25.0,0.0,NaN,12,11289900.0,46111.7,11297300.0,0.0,...,173259.0,4767.63,374307.0,35767.3,250397.0,65966.9,77976.8,486239.0,45032.7,0.00
942_24,2.0,3.0,23.0,NaN,NaN,24,10161900.0,32145.0,12388000.0,25869.2,...,185428.0,5554.53,0.0,64049.8,479473.0,68505.7,74483.1,561398.0,52916.4,21847.60


In [109]:
# Drop categorical feature for now 
# Only keep visits off medication
# df = df.query('upd23b_clinical_state_on_medication != "On"') # Does worse with this
print(df.shape)
df = df.drop('upd23b_clinical_state_on_medication', axis = 1)
df

(1068, 974)


,updrs_1,updrs_2,updrs_3,updrs_4,visit_month,AADDTWEPFASGK,AAFGQGSGPIMLDEVQC(UniMod_4)TGTEASLADC(UniMod_4)K,AAFTEC(UniMod_4)C(UniMod_4)QAADK,AANEVSSADVK,AATGEC(UniMod_4)TATVGKR,...,YSLTYIYTGLSK,YTTEIIK,YVGGQEHFAHLLILR,YVM(UniMod_35)LPVADQDQC(UniMod_4)IR,YVMLPVADQDQC(UniMod_4)IR,YVNKEIQNAVNGVK,YWGVASFLQK,YYC(UniMod_4)FQGNQFLR,YYTYLIMNK,YYWGGQYTWDMAK
visit_id,,,,,,,,,,,,,,,,,,,,,
10053_0,3.0,0.0,13.0,0.0,0,6580710.0,31204.4,7735070.0,0.0,0.00,...,202274.0,0.00,4401830.0,77482.6,583075.0,76705.7,104260.0,530223.0,0.0,7207.30
10053_12,4.0,2.0,8.0,0.0,12,6333510.0,52277.6,5394390.0,0.0,0.00,...,201009.0,0.00,5001750.0,36745.3,355643.0,92078.1,123254.0,453883.0,49281.9,25332.80
10053_18,2.0,2.0,0.0,0.0,18,7129640.0,61522.0,7011920.0,35984.7,17188.00,...,220728.0,0.00,5424380.0,39016.0,496021.0,63203.6,128336.0,447505.0,52389.1,21235.70
10138_12,3.0,6.0,31.0,0.0,12,7404780.0,46107.2,10610900.0,0.0,20910.20,...,188362.0,9433.71,3900280.0,48210.3,328482.0,89822.1,129964.0,552232.0,65657.8,9876.98
10138_24,4.0,7.0,19.0,10.0,24,13788300.0,56910.3,6906160.0,13785.5,11004.20,...,206187.0,6365.15,3521800.0,69984.6,496737.0,80919.3,111799.0,0.0,56977.6,4903.09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8699_24,11.0,10.0,13.0,2.0,24,6312970.0,44462.7,12455000.0,11051.3,1163.18,...,289888.0,8615.27,8770410.0,33599.1,926094.0,118897.0,133682.0,571879.0,80268.3,54889.70
942_12,5.0,2.0,25.0,0.0,12,11289900.0,46111.7,11297300.0,0.0,13894.10,...,173259.0,4767.63,374307.0,35767.3,250397.0,65966.9,77976.8,486239.0,45032.7,0.00
942_24,2.0,3.0,23.0,NaN,24,10161900.0,32145.0,12388000.0,25869.2,17341.80,...,185428.0,5554.53,0.0,64049.8,479473.0,68505.7,74483.1,561398.0,52916.4,21847.60


# Linear Regression Without Feature Selection

In [96]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler

metrics_df = pd.DataFrame(columns=['mse', 'mae', 'r2', 'smape'])

for i in range(1, 5): 
    curr_df = df[[f'updrs_{i}'] + list(df.columns[4:])].dropna()
    print(curr_df.shape)

    X = curr_df.iloc[:, 1:]
    y_updrs = curr_df.iloc[:, 0]

    # Split the dataset into training and testing sets.
    X_train_updrs, X_test_updrs, y_train_updrs, y_test_updrs = train_test_split(X, y_updrs, test_size = 0.2, random_state = 42)

    # Standardize the independent variables.
    scaler = StandardScaler()
    X_train_updrs = scaler.fit_transform(X_train_updrs)
    X_test_updrs = scaler.transform(X_test_updrs)

    # Fit a linear regression model on the training set.
    model_updrs = LinearRegression()
    model_updrs.fit(X_train_updrs, y_train_updrs)

    # Predict the values of the dependent variable (target) on the testing set.
    y_pred_updrs = model_updrs.predict(X_test_updrs)
    y_pred_updrs = np.where(y_pred_updrs < 0, 0, y_pred_updrs)

    # Evaluate the performance of the model.
    mse_updrs = mean_squared_error(y_test_updrs, y_pred_updrs)
    mae_updrs = mean_absolute_error(y_test_updrs, y_pred_updrs)
    r2_updrs = r2_score(y_test_updrs, y_pred_updrs)

    # add mean squared error, mean absolute error, r2 score, and SMAPE for updrs to df 
    metrics_df.loc[f'updrs_{i}'] = [mse_updrs, mae_updrs, r2_updrs, smape(y_test_updrs, y_pred_updrs)]

metrics_df


(839, 970)
(839, 970)
(831, 970)
(345, 970)


,mse,mae,r2,smape
updrs_1,44.899796,5.124628,-0.901045,101.665679
updrs_2,50.335050,5.290332,-0.328635,108.982380
updrs_3,344.949506,14.393606,-0.377277,102.916746
updrs_4,8.035209,2.039711,-0.904213,127.727376


# Feature Selection

Univariate Feature Selection - Pick the most correlated features w target var

In [97]:
from sklearn.feature_selection import SelectKBest, f_regression

metrics_df = pd.DataFrame(columns=['mse', 'mae', 'r2', 'smape'])

for i in range(1, 5): 
    curr_df = df[[f'updrs_{i}'] + list(df.columns[4:])].dropna()
    logger.debug(curr_df.shape)

    X = curr_df.iloc[:, 1:]
    y_updrs = curr_df.iloc[:, 0]

    # Feature Selection 
    X_peptides = X.drop(['visit_month'], axis=1)
    selector = SelectKBest(score_func=f_regression, k=10)
    X_new = selector.fit_transform(X_peptides, y_updrs)
    X = np.concatenate((X_new, X['visit_month'].values.reshape(-1, 1)), axis=1)


    # Split the dataset into training and testing sets.
    X_train_updrs, X_test_updrs, y_train_updrs, y_test_updrs = train_test_split(X, y_updrs, test_size = 0.2, random_state = 42)

    # Standardize the independent variables.
    scaler = StandardScaler()
    X_train_updrs = scaler.fit_transform(X_train_updrs)
    X_test_updrs = scaler.transform(X_test_updrs)

    # Fit a linear regression model on the training set.
    model_updrs = LinearRegression()
    model_updrs.fit(X_train_updrs, y_train_updrs)

    # Predict the values of the dependent variable (target) on the testing set.
    y_pred_updrs = model_updrs.predict(X_test_updrs)
    y_pred_updrs = np.where(y_pred_updrs < 0, 0, y_pred_updrs)

    # Evaluate the performance of the model.
    mse_updrs = mean_squared_error(y_test_updrs, y_pred_updrs)
    mae_updrs = mean_absolute_error(y_test_updrs, y_pred_updrs)
    r2_updrs = r2_score(y_test_updrs, y_pred_updrs)

    # add mean squared error, mean absolute error, r2 score, and SMAPE for updrs to df 
    metrics_df.loc[f'updrs_{i}'] = [mse_updrs, mae_updrs, r2_updrs, smape(y_test_updrs, y_pred_updrs)]

metrics_df

,mse,mae,r2,smape
updrs_1,22.594070,3.621339,0.043373,70.859616
updrs_2,33.436381,4.349308,0.117419,105.686520
updrs_3,246.088952,13.608993,0.017443,104.353176
updrs_4,3.963247,1.483773,0.060776,138.761503


Recursive Feature Elimination - iteratively remove the least useful feature

In [66]:
from sklearn.feature_selection import RFE 

metrics_df = pd.DataFrame(columns=['mse', 'mae', 'r2', 'smape'])

for i in range(1, 5): 
    logger.info(f'updrs_{i}')
    curr_df = df[[f'updrs_{i}'] + list(df.columns[4:])].dropna()
    logger.debug(curr_df.shape)

    X = curr_df.iloc[:, 1:]
    y_updrs = curr_df.iloc[:, 0]

    model_updrs = LinearRegression()

    # Split the dataset into training and testing sets.
    X_train_updrs, X_test_updrs, y_train_updrs, y_test_updrs = train_test_split(X, y_updrs, test_size = 0.2, random_state = 42)

    # Standardize the independent variables.
    scaler = StandardScaler()
    X_train_updrs = scaler.fit_transform(X_train_updrs)
    X_test_updrs = scaler.transform(X_test_updrs)

    # Feature Selection - on training data only 
    rfe = RFE(model_updrs, n_features_to_select=5, step=1)
    X_train_updrs = rfe.fit_transform(X_train_updrs, y_train_updrs)
    X_test_updrs = rfe.transform(X_test_updrs)

    # Fit a linear regression model on the training set.
    model_updrs.fit(X_train_updrs, y_train_updrs)

    # Predict the values of the dependent variable (target) on the testing set.
    y_pred_updrs = model_updrs.predict(X_test_updrs)
    y_pred_updrs = np.where(y_pred_updrs < 0, 0, y_pred_updrs)

    # Evaluate the performance of the model.
    mse_updrs = mean_squared_error(y_test_updrs, y_pred_updrs)
    mae_updrs = mean_absolute_error(y_test_updrs, y_pred_updrs)
    r2_updrs = r2_score(y_test_updrs, y_pred_updrs)

    # add mean squared error, mean absolute error, r2 score, and SMAPE for updrs to df 
    metrics_df.loc[f'updrs_{i}'] = [mse_updrs, mae_updrs, r2_updrs, smape(y_test_updrs, y_pred_updrs)]

metrics_df

INFO:__main__:updrs_1
INFO:__main__:updrs_2
INFO:__main__:updrs_3
INFO:__main__:updrs_4


,mse,mae,r2,smape
updrs_1,23.914877,4.092584,-0.087043,76.198527
updrs_2,37.363506,4.839796,-0.049594,105.168949
updrs_3,240.321916,12.910960,0.032759,97.828539
updrs_4,8.380497,2.276784,-0.135113,154.403605


PCA + K best selection

In [74]:
from sklearn.decomposition import PCA

metrics_df = pd.DataFrame(columns=['mse', 'mae', 'r2', 'smape'])

for i in range(1, 5): 
    logger.info(f'updrs_{i}')
    curr_df = df[[f'updrs_{i}'] + list(df.columns[4:])].dropna()
    logger.debug(curr_df.shape)

    X = curr_df.iloc[:, 1:]
    y_updrs = curr_df.iloc[:, 0]

    pca = PCA(n_components=50)
    X = pd.DataFrame(pca.fit_transform(X))

    # Feature Selection 
    selector = SelectKBest(score_func=f_regression, k=10)
    X = selector.fit_transform(X, y_updrs)

    # Split the dataset into training and testing sets.
    X_train_updrs, X_test_updrs, y_train_updrs, y_test_updrs = train_test_split(X, y_updrs, test_size = 0.2, random_state = 42)

    # Standardize the independent variables.
    scaler = StandardScaler()
    X_train_updrs = scaler.fit_transform(X_train_updrs)
    X_test_updrs = scaler.transform(X_test_updrs)

    # Fit a linear regression model on the training set.
    model_updrs = LinearRegression()
    model_updrs.fit(X_train_updrs, y_train_updrs)

    # Predict the values of the dependent variable (target) on the testing set.
    y_pred_updrs = model_updrs.predict(X_test_updrs)
    y_pred_updrs = np.where(y_pred_updrs < 0, 0, y_pred_updrs)

    # Evaluate the performance of the model.
    mse_updrs = mean_squared_error(y_test_updrs, y_pred_updrs)
    mae_updrs = mean_absolute_error(y_test_updrs, y_pred_updrs)
    r2_updrs = r2_score(y_test_updrs, y_pred_updrs)

    # add mean squared error, mean absolute error, r2 score, and SMAPE for updrs to df 
    metrics_df.loc[f'updrs_{i}'] = [mse_updrs, mae_updrs, r2_updrs, smape(y_test_updrs, y_pred_updrs)]

metrics_df

INFO:__main__:updrs_1
INFO:__main__:updrs_2
INFO:__main__:updrs_3
INFO:__main__:updrs_4


,mse,mae,r2,smape
updrs_1,23.354430,4.016368,-0.061568,75.599832
updrs_2,35.177875,4.756695,0.011803,104.209917
updrs_3,244.087704,13.014068,0.017603,96.534643
updrs_4,7.121574,2.202916,0.035405,152.736323


Add Lasso and Ridge to above

In [75]:
from sklearn.linear_model import Lasso, Ridge

metrics_df = pd.DataFrame(columns=['mse', 'mae', 'r2', 'smape'])

for i in range(1, 5): 
    logger.info(f'updrs_{i}')
    curr_df = df[[f'updrs_{i}'] + list(df.columns[4:])].dropna()
    logger.debug(curr_df.shape)

    X = curr_df.iloc[:, 1:]
    y_updrs = curr_df.iloc[:, 0]

    pca = PCA(n_components=50)
    X = pd.DataFrame(pca.fit_transform(X))

    # Feature Selection 
    selector = SelectKBest(score_func=f_regression, k=10)
    X = selector.fit_transform(X, y_updrs)

    # Split the dataset into training and testing sets.
    X_train_updrs, X_test_updrs, y_train_updrs, y_test_updrs = train_test_split(X, y_updrs, test_size = 0.2, random_state = 42)

    # Standardize the independent variables.
    scaler = StandardScaler()
    X_train_updrs = scaler.fit_transform(X_train_updrs)
    X_test_updrs = scaler.transform(X_test_updrs)

    # Fit a linear regression model on the training set.
    lasso = Lasso(alpha=0.1)
    lasso.fit(X_train_updrs, y_train_updrs)

    ridge = Ridge(alpha=0.1)
    ridge.fit(X_train_updrs, y_train_updrs)

    # Predict the values of the dependent variable (target) on the testing set.
    y_pred_lasso = lasso.predict(X_test_updrs)
    y_pred_lasso = np.where(y_pred_lasso < 0, 0, y_pred_lasso)

    y_pred_ridge = ridge.predict(X_test_updrs)
    y_pred_ridge = np.where(y_pred_ridge < 0, 0, y_pred_ridge)

    # Evaluate the performance of the model.
    mse_lasso = mean_squared_error(y_test_updrs, y_pred_lasso)
    mae_lasso = mean_absolute_error(y_test_updrs, y_pred_lasso)
    r2_lasso = r2_score(y_test_updrs, y_pred_lasso)

    mse_ridge = mean_squared_error(y_test_updrs, y_pred_ridge)
    mae_ridge = mean_absolute_error(y_test_updrs, y_pred_ridge)
    r2_ridge = r2_score(y_test_updrs, y_pred_ridge)

    # add mean squared error, mean absolute error, r2 score, and SMAPE for updrs to df
    metrics_df.loc[f'updrs_{i}_lasso'] = [mse_lasso, mae_lasso, r2_lasso, smape(y_test_updrs, y_pred_lasso)]
    metrics_df.loc[f'updrs_{i}_ridge'] = [mse_ridge, mae_ridge, r2_ridge, smape(y_test_updrs, y_pred_ridge)]


metrics_df

INFO:__main__:updrs_1
INFO:__main__:updrs_2
INFO:__main__:updrs_3
INFO:__main__:updrs_4


,mse,mae,r2,smape
updrs_1_lasso,23.249869,4.016478,-0.056815,75.609412
updrs_1_ridge,23.354776,4.016432,-0.061584,75.600297
updrs_2_lasso,35.156373,4.766656,0.012407,103.805442
updrs_2_ridge,35.177757,4.756698,0.011806,104.209599
updrs_3_lasso,243.843241,13.017606,0.018587,96.449293
updrs_3_ridge,244.086177,13.014042,0.017609,96.534468
updrs_4_lasso,7.223857,2.233878,0.021551,152.460204
updrs_4_ridge,7.121498,2.202918,0.035415,152.735587


Adding Random Forest Regression

In [98]:
from sklearn.ensemble import RandomForestRegressor

metrics_df = pd.DataFrame(columns=['mse', 'mae', 'r2', 'smape'])

for i in range(1, 5): 
    logger.info(f'updrs_{i}')
    curr_df = df[[f'updrs_{i}'] + list(df.columns[4:])].dropna()
    logger.debug(curr_df.shape)

    X = curr_df.iloc[:, 1:]
    y_updrs = curr_df.iloc[:, 0]

    pca = PCA(n_components=50)
    X = pd.DataFrame(pca.fit_transform(X))

    # Feature Selection 
    selector = SelectKBest(score_func=f_regression, k=10)
    X = selector.fit_transform(X, y_updrs)

    # Split the dataset into training and testing sets.
    X_train_updrs, X_test_updrs, y_train_updrs, y_test_updrs = train_test_split(X, y_updrs, test_size = 0.2, random_state = 42)

    # Standardize the independent variables.
    scaler = StandardScaler()
    X_train_updrs = scaler.fit_transform(X_train_updrs)
    X_test_updrs = scaler.transform(X_test_updrs)

    # Fit a random forest regressor model on the training set.
    rf = RandomForestRegressor(n_estimators=100, random_state=42)
    rf.fit(X_train_updrs, y_train_updrs)

    # Predict the values of the dependent variable (target) on the testing set.
    y_pred_rf = rf.predict(X_test_updrs)
    y_pred_rf = np.where(y_pred_rf < 0, 0, y_pred_rf)

    # Evaluate the performance of the model.
    mse_rf = mean_squared_error(y_test_updrs, y_pred_rf)
    mae_rf = mean_absolute_error(y_test_updrs, y_pred_rf)
    r2_rf = r2_score(y_test_updrs, y_pred_rf)

    # add mean squared error, mean absolute error, r2 score, and SMAPE for updrs to df
    metrics_df.loc[f'updrs_{i}_rf'] = [mse_rf, mae_rf, r2_rf, smape(y_test_updrs, y_pred_rf)]    

metrics_df

INFO:__main__:updrs_1
INFO:__main__:updrs_2
INFO:__main__:updrs_3
INFO:__main__:updrs_4


,mse,mae,r2,smape
updrs_1_rf,23.042536,3.804821,0.024385,72.933662
updrs_2_rf,33.273592,4.491369,0.121716,107.530850
updrs_3_rf,217.812827,12.858144,0.130341,103.775771
updrs_4_rf,5.270322,1.766957,-0.248980,168.435208


# Trying with protein dataset

In [80]:
merged_proteins_peptides = pd.merge(proteins, peptides, on = ['visit_id', 'visit_month', 'patient_id', 'UniProt'])
merged = pd.merge(merged_proteins_peptides, clinical, on = ['visit_id', 'visit_month', 'patient_id'])
merged['Peptide / Protein'] = merged['PeptideAbundance'] / merged['NPX']

merged

,visit_id,visit_month,patient_id,UniProt,NPX,Peptide,PeptideAbundance,updrs_1,updrs_2,updrs_3,updrs_4,upd23b_clinical_state_on_medication,Peptide / Protein
0,55_0,0,55,O00391,11254.3,NEQEQPLGQWHLS,11254.30,10.0,6.0,15.0,NaN,NaN,1.000000
1,55_0,0,55,O00533,732430.0,GNPEPTFSWTK,102060.00,10.0,6.0,15.0,NaN,NaN,0.139344
2,55_0,0,55,O00533,732430.0,IEIPSSVQQVPTIIK,174185.00,10.0,6.0,15.0,NaN,NaN,0.237818
3,55_0,0,55,O00533,732430.0,KPQSAVYSTGSNGILLC(UniMod_4)EAEGEPQPTIK,27278.90,10.0,6.0,15.0,NaN,NaN,0.037244
4,55_0,0,55,O00533,732430.0,SMEQNGPGLEYR,30838.70,10.0,6.0,15.0,NaN,NaN,0.042105
...,...,...,...,...,...,...,...,...,...,...,...,...,...
941739,58648_108,108,58648,Q9UHG2,369437.0,ILAGSADSEGVAAPR,202820.00,6.0,0.0,0.0,NaN,NaN,0.548998
941740,58648_108,108,58648,Q9UKV8,105830.0,SGNIPAGTTVDTK,105830.00,6.0,0.0,0.0,NaN,NaN,1.000000
941741,58648_108,108,58648,Q9Y646,21257.6,LALLVDTVGPR,21257.60,6.0,0.0,0.0,NaN,NaN,1.000000
941742,58648_108,108,58648,Q9Y6R7,17953.1,AGC(UniMod_4)VAESTAVC(UniMod_4)R,5127.26,6.0,0.0,0.0,NaN,NaN,0.285592


In [85]:
merged_dd = merged.iloc[:, :5].drop_duplicates(subset = ['visit_id', 'visit_month', 'patient_id', 'UniProt'])
pivoted = merged_dd.pivot(index = 'visit_id', columns = ['UniProt'], values = 'NPX')
df = pd.merge(clinical, pivoted, on = 'visit_id', how = 'right').set_index('visit_id')

df

,patient_id,visit_month,updrs_1,updrs_2,updrs_3,updrs_4,upd23b_clinical_state_on_medication,O00391,O00533,O00584,...,Q9HDC9,Q9NQ79,Q9NYU2,Q9UBR2,Q9UBX5,Q9UHG2,Q9UKV8,Q9UNU6,Q9Y646,Q9Y6R7
visit_id,,,,,,,,,,,,,,,,,,,,,
10053_0,10053,0,3.0,0.0,13.0,0.0,NaN,9104.27,402321.0,NaN,...,NaN,9469.45,94237.6,NaN,23016.0,177983.0,65900.0,15382.0,NaN,19017.40
10053_12,10053,12,4.0,2.0,8.0,0.0,NaN,10464.20,435586.0,NaN,...,NaN,14408.40,NaN,NaN,28537.0,171733.0,65668.1,NaN,9295.65,25697.80
10053_18,10053,18,2.0,2.0,0.0,0.0,NaN,13235.70,507386.0,7126.96,...,317477.0,38667.20,111107.0,NaN,37932.6,245188.0,59986.1,10813.3,NaN,29102.70
10138_12,10138,12,3.0,6.0,31.0,0.0,On,12600.20,494581.0,9165.06,...,557904.0,44556.90,155619.0,14647.90,36927.7,229232.0,106564.0,26077.7,21441.80,7642.42
10138_24,10138,24,4.0,7.0,19.0,10.0,On,12003.20,522138.0,4498.51,...,NaN,47836.70,177619.0,17061.10,25510.4,176722.0,59471.4,12639.2,15091.40,6168.55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8699_24,8699,24,11.0,10.0,13.0,2.0,On,9983.00,400290.0,24240.10,...,NaN,25690.60,NaN,6859.82,19106.7,121161.0,113872.0,14413.9,28225.50,8062.07
942_12,942,12,5.0,2.0,25.0,0.0,NaN,6757.32,360858.0,18367.60,...,45742.3,33518.60,94049.7,13415.70,21324.7,234094.0,82410.4,19183.7,17804.10,12277.00
942_24,942,24,2.0,3.0,23.0,NaN,NaN,NaN,352722.0,22834.90,...,180475.0,29770.60,95949.9,11344.40,23637.6,256654.0,76931.9,19168.2,19215.90,14625.60
